In [ ]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import plotly.graph_objects as go
import json

# Creating Brazil-Specific Maps for Wildfires and Birds

In [ ]:
# Read in Brazil fire data
brazil_fire = Path('resources/brazil_fires_by_state.csv')
brazil_df = pd.read_csv(brazil_fire, encoding='ISO-8859-1')

# find unique state name for proper abbrevations
states = brazil_df['state']
states.unique()


In [ ]:
# rename states to 2-letter code so that data can merge
state_rename_mapping = {
    'Acre' : "AC",
    'Alagoas' : "AL",
    'Amapa' : "AP", 
    'Amazonas' : "AM", 
    'Bahia' : "BA", 
    'Ceara' : "CE",
    'Distrito Federal' : "DF", 
    'Espirito Santo' : "ES", 
    'Goias' : "GO", 
    'Maranhao' : "MA",
    'Mato Grosso' : "MT", 
    'Minas Gerais' : "MG", 
    'Pará' : "PA", 
    'Paraiba' :"PB", 
    'Pernambuco' : "PE",
    'Piau' : "PI",
    'Rio' : "RJ", 
    'Rondonia' : "RO",
    'Roraima' : "RR", 
    'Santa Catarina' : "SC",
    'Sao Paulo' : "SP", 
    'Sergipe' : "SE", 
    'Tocantins' : "TO"
}

In [ ]:
# rename state column values with abbrevations for each
brazil_df['state'] = brazil_df['state'].replace(state_rename_mapping)
brazil_df.head()

In [ ]:
# rename column label
brazil_abbrev = brazil_df.rename(columns = {
 "state" : "state_abbr"
})

brazil_abbrev.head()

In [ ]:
# group by year and state to find total reported fires for 2013
brazil_fire_2013 = brazil_abbrev[brazil_abbrev['year'] == 2013]
group_2013 = brazil_fire_2013.groupby(['year', 'state_abbr'])['number'].sum().reset_index()

group_2013.head()

In [ ]:
# repeat above for 2017
brazil_fire_2017 = brazil_abbrev[brazil_abbrev['year'] == 2017]
group_2017 = brazil_fire_2017.groupby(['year', 'state_abbr'])['number'].sum().reset_index()

group_2017.head()

In [ ]:
# load json file with state geometry
with open('resources/brazil-states.geojson') as f:
    json_data = json.load(f)

# Convert to GeoDataFrame with GeoPandas
gdf = gpd.GeoDataFrame.from_features(json_data['features'])

gdf.head()

In [ ]:
gdf = gdf.rename(columns = {
    'sigla' : 'state_abbr'
})

gdf.head()

## Merging Brazil data with Geojson coordinates for mapping

In [ ]:
# merge 2013 and 2017 data with the json file
gdf_2013 = gdf.merge(brazil_fire_2013, how='left', on='state_abbr')
gdf_2017 = gdf.merge(brazil_fire_2017, how='left', on='state_abbr')

gdf_2013.head()

# 2013 Wildfire and Bird Map

In [ ]:
# Load csv
birds_2013 = Path('resources/brazil_birds_2013.csv')
bird_path_2013 = pd.read_csv(birds_2013)

# Filter relevant columns
bird_path_2013_filtered = bird_path_2013[['decimalLatitude', 'decimalLongitude', 'species', 'month']]

# Remove rows with missing latitude or longitude values
bird_path_2013_filtered = bird_path_2013_filtered.dropna(subset=['decimalLatitude', 'decimalLongitude'])
bird_path_2013_filtered = bird_path_2013_filtered.sort_values(by='month')
# Group by species and month to calculate mean latitudes and longitudes
bird_path_2013_grouped = bird_path_2013_filtered.groupby(['species', 'month'], as_index=False).agg({
    'decimalLatitude': 'mean',
    'decimalLongitude': 'mean'
})
bird_path_2013_grouped['month'] = bird_path_2013_grouped['month'].astype(int)
bird_path_2013_grouped = bird_path_2013_grouped.sort_values(by='month')

# Determine direction of movement based on latitude difference
bird_path_2013_grouped['direction'] = bird_path_2013_grouped.groupby('species')['decimalLatitude'].diff()

# Assign colors based on direction (upward = green, downward = blue)
bird_path_2013_grouped['color'] = bird_path_2013_grouped['direction'].apply(lambda x: 'green' if x > 0 else 'blue')


In [ ]:
# Make 2013 Choropleth map
fig_2013 = px.choropleth(
    gdf_2013,
    geojson=gdf_2013.geometry,
    locations=gdf_2013.index,
    color='number',
    hover_name='name',
    hover_data=['number'],
    color_continuous_scale="Reds",
    title="Wildfires in Brazil - 2013", 
    height = 400, 
    width = 600
)
fig_2013.update_geos(fitbounds="locations", visible=False)

scatter_trace_2013 = go.Scattergeo(
    lat=bird_path_2013_grouped['decimalLatitude'],
    lon=bird_path_2013_grouped['decimalLongitude'],
    mode='markers',
    marker=dict(size=6, color='blue', symbol='circle'),
    text=bird_path_2013_grouped['species'],  # Display species on hover
    name='Birds',  # Name for the scatter layer
)

# Step 4: Add Scatter Trace to Choropleth Map
fig_2013.add_trace(scatter_trace_2013)

# Step 5: Display the Combined Map
fig_2013.update_layout(
    geo=dict(
        projection_type='mercator',  # or use 'natural earth'
        scope='south america',  # Limit the map view to South America (Brazil in particular)
    ),
    title_text="Wildfires in Brazil - 2013 with Bird Data",  # Title for the map
    title_x=0.5,  # Center the title
    title_y=0.98,
    margin={"r":0,"t":0,"l":0,"b":0}, 
    height = 400, 
    width = 600 
)
fig_2013.write_html("html/brazil_birds_2013.html")
fig_2013.show()

# 2017 Wildfire and Bird Map

In [ ]:
# Load csv
birds_2017 = Path('resources/brazil_birds_2017.csv')
bird_path_2017 = pd.read_csv(birds_2017)

# Filter relevant columns
bird_path_2017_filtered = bird_path_2017[['decimalLatitude', 'decimalLongitude', 'species', 'month']]

# Remove rows with missing latitude or longitude values
bird_path_2017_filtered = bird_path_2017_filtered.dropna(subset=['decimalLatitude', 'decimalLongitude'])
bird_path_2017_filtered = bird_path_2017_filtered.sort_values(by='month')
# Group by species and month to calculate mean latitudes and longitudes
bird_path_2017_grouped = bird_path_2017_filtered.groupby(['species', 'month'], as_index=False).agg({
    'decimalLatitude': 'mean',
    'decimalLongitude': 'mean'
})
bird_path_2017_grouped['month'] = bird_path_2017_grouped['month'].astype(int)
bird_path_2017_grouped = bird_path_2017_grouped.sort_values(by='month')

# Determine direction of movement based on latitude difference
bird_path_2017_grouped['direction'] = bird_path_2017_grouped.groupby('species')['decimalLatitude'].diff()

# Assign colors based on direction (upward = green, downward = blue)
bird_path_2017_grouped['color'] = bird_path_2017_grouped['direction'].apply(lambda x: 'green' if x > 0 else 'blue')

In [ ]:
# Make 2017 Choropleth map
fig_2017 = px.choropleth(
    gdf_2017,
    geojson=gdf_2017.geometry,
    locations=gdf_2017.index,
    color='number',
    hover_name='name',
    hover_data=['number'],
    color_continuous_scale="Reds",
    title="Wildfires in Brazil - 2017", 
    height = 400, 
    width = 600
)
fig_2017.update_geos(fitbounds="locations", visible=False) 

scatter_trace_2017 = go.Scattergeo(
    lat=bird_path_2017_grouped['decimalLatitude'],
    lon=bird_path_2017_grouped['decimalLongitude'],
    mode='markers',
    marker=dict(size=6, color='blue', symbol='circle'),
    text=bird_path_2017_grouped['species'],  # Display species on hover
    name='Birds',  # Name for the scatter layer
)

# Step 4: Add Scatter Trace to Choropleth Map
fig_2017.add_trace(scatter_trace_2017)

# Step 5: Display the Combined Map
fig_2017.update_layout(
    geo=dict(
        projection_type='mercator',  # or use 'natural earth'
        scope='south america',  # Limit the map view to South America (Brazil in particular)
    ),
    title_text="Wildfires in Brazil - 2017 with Bird Data",  # Title for the map
    title_x=0.5,  # Center the title
    title_y=0.98,
    margin={"r":0,"t":0,"l":0,"b":0}, 
    height = 400, 
    width = 600 
)
fig_2017.write_html("html/brazil_birds_2017.html")
fig_2017.show()